# ECE 4194.04 Final Project - Citation Thing
  __Authors:__  Omar Mansour, Shreyas Murlaidharan, Ryan Weiper

## Part 0: Data pre-processing

The dataset is in a bastardized JSON format. We first iterate over it to put it into a pandas dataframe

In [29]:
import pandas as pd
import numpy as np

# Open the data
#data_str = open('acm1st10000.txt',errors="ignore").read()     # small data
data_str = open('acm.txt',encoding="utf8").read()  # full dataset

data_list = data_str.splitlines() # split by line breaks

# Create the dataframe
    # title: (string -- should be bag of words vector)
    # authors: (list)
    # year: publishing year (int)
    # venue: journal the paper was published in (string)
    # ind: numeric label/index of this paper (int)
    # ref_indces: identification index of the papers that referenced this paper (string)
    # abstract: (string -- should be bag of words vector ?)
names = ['title', 'authors', 'year', 'venue', 'abstract','citations']

title = "NULL"
authors = "NULL"
year = "NULL"
venue = "NULL"
index = 0
abstract = "NULL"
citations = 0

df_list = []

for i in range(len(data_list)):
    if(i%10**6 == 0):  # progress update 
        print(i)
    prefix = data_list[i][0:2]
    if prefix == "#*": # title
        title = data_list[i][2:]
        continue
    if prefix == "#@": #author list
        authors = data_list[i][2:]
        continue
    if prefix == "#t": # year
        year = data_list[i][2:]
        continue
    if prefix == "#c": #journal
        venue = data_list[i][2:]
        continue
    if prefix == "#!": #abstract
        abstract = data_list[i][2:]
    if prefix == "#%": #paper citation
        citations += 1
    if data_list[i] == "": #check for the empty string (new article)       
        if year=='2015':
            df_list.append([title, authors, year, venue, abstract,citations])
            # print(index,citations) #debug
            title = "NULL"
            authors = "NULL"
            year = "NULL"
            venue = "NULL"
            abstract = "NULL"
            
            index += 1
        citations = 0

print("Done")

df = pd.DataFrame(columns=names)
df = pd.DataFrame.from_records(df_list, columns=names)
df.head(10)

    

# T0-DO:
# -----------------------------------------------------------
# Force strings in dataframe that we want as ints to be ints
# Convert the strings in dataframe to bag of words vectors 
# Make a dataframe of just 2015 papers

# Account for no citations in a dataframe (if citation column is NaN, make it 0)
# Account for no abstract (if abstract column is NaN, remove the row completely)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
Done


,title,authors,year,venue,abstract,citations
0,Finding an Initial Basic Feasible Solution for...,"Mehdi Toloo, Atefeh Masoumzadeh, Mona Barat",2015,Computational Economics,"Nowadays, algorithms and computer programs, wh...",3
1,Tag line generating system using knowledge ext...,"Hiroaki Yamane, Masafumi Hagiwara",2015,AI & Society,This paper proposes a tag line generating syst...,2
2,Computational Economics,NULL,2015,Computational Economics,NULL,0
3,A social contract for virtual institutions,Daniel Memmi,2015,AI & Society,"Computer-mediated social groups, often known a...",6
4,AI & Society,NULL,2015,AI & Society,NULL,0
5,Solving Dynamic Programming Problems on a Comp...,"Yongyang Cai, Kenneth L. Judd, Greg Thain, Ste...",2015,Computational Economics,We implement a dynamic programming algorithm o...,8
6,ESIS2: Information Value Estimator for Credit ...,Martin Řezáč,2015,Computational Economics,Information value is widely used to assess dis...,5
7,Social media analytics: a survey of techniques...,"Bogdan Batrinca, Philip C. Treleaven",2015,AI & Society,This paper is written for (social science) res...,9
8,Carbon Price Analysis Using Empirical Mode Dec...,"Bangzhu Zhu, Ping Wang, Julien Chevallier, Yim...",2015,Computational Economics,Mastering the underlying characteristics of ca...,4
9,Crime detection and criminal identification in...,"Devendra Kumar Tayal, Arti Jain, Surbhi Arora,...",2015,AI & Society,"In the current paper, we propose an approach f...",9


## Part 1: Aggregate and cluster?

blah blah

In [49]:
#print(df.head(5))
#print(newdf)

#print(list_2k15_Y)


#print list of journals
list_journals = df['venue'].unique()
list_authors  = df['authors'].unique()
print("There are %s unique journal names :)" % len(list_journals))
print("There are %s unique author names :)" % len(list_authors))



There are 1099 unique journal names :)
There are 23239 unique author names :)


In [53]:
#make a dictionary or author name and citations
# key is author name
# value is list: [number of citations,number papers]
    # (avg citation count can be computer as author_dict["name"][0]/author_dict["name"][1])
author_dict = {}
    
# loop over data frame
for i in range(len(df.index)): 
    for author in df.iloc[i][1].split(","):
        a = author.strip()
        cit = df.iloc[i][5]   # citation count
        if a in author_dict:   #author already is in dictionary
            author_dict[a] = [author_dict[a][0]+cit,author_dict[a][1]+1]
        else:                 # author not in dictionary
            author_dict[a] = [cit,1]
author_avg_citations_dict = {}
for key in author_dict:
    author_avg_citations_dict[key] = author_dict[key][0]/author_dict[key][1]

# test code
print("Mehdi Toloo avg citations", author_avg_citations_dict['Mehdi Toloo']) #should be 5/3


# Create a list of author scores to replace the df['authors'] column
# author score is simply the mean of the avg citations counts for each author
authorscore_list = []
for i in range(len(df.index)): 
    score = 0
    authors = df.iloc[i][1].split(",")
    for author in authors:
        score += author_avg_citations_dict[author.strip()]
    authorscore_list.append(score/len(authors))

# Add column of author score to df
df['author score'] = authorscore_list


Mehdi Toloo avg citations 1.6666666666666667


In [57]:
df.head(6)

,title,authors,year,venue,abstract,citations,author score
0,Finding an Initial Basic Feasible Solution for...,"Mehdi Toloo, Atefeh Masoumzadeh, Mona Barat",2015,Computational Economics,"Nowadays, algorithms and computer programs, wh...",3,2.555556
1,Tag line generating system using knowledge ext...,"Hiroaki Yamane, Masafumi Hagiwara",2015,AI & Society,This paper proposes a tag line generating syst...,2,1.000000
2,Computational Economics,NULL,2015,Computational Economics,NULL,0,0.472522
3,A social contract for virtual institutions,Daniel Memmi,2015,AI & Society,"Computer-mediated social groups, often known a...",6,3.000000
4,AI & Society,NULL,2015,AI & Society,NULL,0,0.472522
5,Solving Dynamic Programming Problems on a Comp...,"Yongyang Cai, Kenneth L. Judd, Greg Thain, Ste...",2015,Computational Economics,We implement a dynamic programming algorithm o...,8,7.125000


In [4]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreyasmuralidharan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
list_2k15_X=df['title'].tolist()
author_scorelist_2k15=df['author score'].values
list_2k15_y=df['citations'].tolist()
ntot = len(list_2k15_X)
print(ntot)
from sklearn.feature_extraction.text import CountVectorizer
set_to_add=set([])
stop_words_NLTK = set(stopwords.words('english'))
total_stopwords=stop_words_NLTK|set_to_add
BOWvectorizer = CountVectorizer(stop_words=total_stopwords, min_df=0.0001, max_df=0.99, max_features=2000)
X_bow = BOWvectorizer.fit_transform(list_2k15_X).toarray()
X_bow=np.concatenate((X_bow,author_scorelist_2k15.reshape(26921,1)),axis=1)

y = list_2k15_y
print(len(BOWvectorizer.get_feature_names()))


26921
2000


,title,authors,year,venue,abstract,citations,author score
0,Finding an Initial Basic Feasible Solution for...,"Mehdi Toloo, Atefeh Masoumzadeh, Mona Barat",2015,Computational Economics,"Nowadays, algorithms and computer programs, wh...",3,2.555556
1,Tag line generating system using knowledge ext...,"Hiroaki Yamane, Masafumi Hagiwara",2015,AI & Society,This paper proposes a tag line generating syst...,2,1.000000
2,Computational Economics,NULL,2015,Computational Economics,NULL,0,0.472522
3,A social contract for virtual institutions,Daniel Memmi,2015,AI & Society,"Computer-mediated social groups, often known a...",6,3.000000
4,AI & Society,NULL,2015,AI & Society,NULL,0,0.472522
5,Solving Dynamic Programming Problems on a Comp...,"Yongyang Cai, Kenneth L. Judd, Greg Thain, Ste...",2015,Computational Economics,We implement a dynamic programming algorithm o...,8,7.125000


## Part 2: Model and Predictions

heres the actual ML

In [96]:
ntr = ntot // 2
nts = ntot - ntr

Xtr = X_bow[:ntr,:]
ytr = y[:ntr]
Xts = X_bow[ntr:,:]
yts = y[ntr:]



import sklearn.linear_model
from sklearn.preprocessing import PolynomialFeatures

# TODO

# Create linear regression object
regr = sklearn.linear_model.LinearRegression()

# Fit the model
regr.fit(Xtr,ytr)



# predict
yts_hat = regr.predict(Xts)

#for i in range(100):
#    print(yts[i],yts_hat[i])
    
RSSts = np.mean((yts-yts_hat)**2)
print("rsquared",regr.score(Xts,yts))
print("RSS", RSSts)


rsquared 0.8562540019567663
RSS 12.78064129316212


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDFVectorizer=TfidfVectorizer(stop_words=total_stopwords, min_df=0.01, max_df=0.9, max_features=100)
X_TFIDF = TFIDFVectorizer.fit_transform(list_2k15_X)
print(TFIDFVectorizer.get_feature_names())

ntr = ntot // 2
nts = ntot - ntr

Xtr = X_TFIDF[:ntr,:]
ytr = y[:ntr]
Xts = X_TFIDF[ntr:,:]
yts = y[ntr:]



import sklearn.linear_model

# TODO

# Create linear regression object
regr = sklearn.linear_model.LinearRegression()

# Fit the model
regr.fit(Xtr,ytr)



# predict
yts_hat = regr.predict(Xts)

for i in range(100):
    print(yts[i],yts_hat[i])




['acm', 'adaptive', 'algorithm', 'algorithms', 'analysis', 'application', 'applications', 'approach', 'aware', 'board', 'case', 'classification', 'cloud', 'communication', 'computer', 'computing', 'conference', 'control', 'data', 'design', 'detection', 'development', 'digital', 'distributed', 'dynamic', 'editorial', 'efficient', 'energy', 'evaluation', 'framework', 'fuzzy', 'high', 'human', 'hybrid', 'image', 'information', 'interaction', 'international', 'journal', 'knowledge', 'large', 'learning', 'linear', 'management', 'method', 'methods', 'mobile', 'model', 'modeling', 'models', 'multi', 'multiple', 'network', 'networks', 'new', 'non', 'online', 'optimal', 'optimization', 'order', 'performance', 'power', 'problem', 'problems', 'proceedings', 'programming', 'real', 'recognition', 'research', 'scale', 'scheduling', 'science', 'search', 'security', 'sensor', 'service', 'simulation', 'social', 'software', 'support', 'technology', 'time', 'two', 'user', 'via', 'web', 'wireless']
1 1.69

# Part 3: Clustering

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
TFIDFVectorizer=TfidfVectorizer(stop_words=total_stopwords, min_df=0.001, max_df=0.999, max_features=1000)
X_TFIDF = TFIDFVectorizer.fit_transform(list_2k15_X)
print(len(TFIDFVectorizer.get_feature_names()))
idx_to_word = np.array(TFIDFVectorizer.get_feature_names())
#print(idx_to_word)
nmf = NMF(n_components=20, solver="mu")
W = nmf.fit_transform(X_TFIDF)
H = nmf.components_
for i, topic in enumerate(H):
 
    print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))

print(nmf.get_params)

1000
Topic 1: abstracts,2015,international,33rd,factors,acm,annual,human,conference,proceedings
Topic 2: changes,page,issue,special,guest,cover,front,inside,board,editorial
Topic 3: fuzzy,detection,network,new,efficient,problems,problem,optimization,method,algorithm
Topic 4: remote,research,digital,intelligent,technology,mathematics,computational,sciences,international,journal
Topic 5: privacy,cloud,clustering,open,analytics,driven,web,mining,big,data
Topic 6: communication,technologies,processing,knowledge,retrieval,management,issue,security,technology,information
Topic 7: review,theoretical,abstract,acm,46th,technical,symposium,education,science,computer
Topic 8: models,user,optimal,implementation,europe,exhibition,test,2015,automation,design
Topic 9: network,ad,hoc,protocol,energy,routing,neural,sensor,wireless,networks
Topic 10: framework,stability,impact,online,research,case,computational,modeling,performance,analysis
Topic 11: supervised,scale,language,classification,via,knowledg